In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [46]:
df = pd.read_csv('manipal_sah_tr.csv')
df_test = pd.read_csv('manipal_sah_test.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 9 columns):
Age                           627 non-null int64
Sex                           627 non-null int64
Hypertension                  627 non-null int64
Daibetes                      627 non-null int64
WFNS grade                    627 non-null int64
Radiological Vasospas1        627 non-null int64
Clinical ischemic de0icits    627 non-null int64
Outcome at Discharge          627 non-null int64
Outcome at 3 months           627 non-null int64
dtypes: int64(9)
memory usage: 44.2 KB


In [47]:
print(df.head())
#print(df_test.head())
df.drop(['Radiological Vasospas1', 'Clinical ischemic de0icits', 'Outcome at Discharge'], inplace = True,  axis = 1) 
#df.drop(df.iloc[:, 5:8], inplace = True, axis = 1) 

#df_test.drop(['Radiological Vasospas1', 'Clinical ischemic de0icits', 'Outcome at Discharge'], axis = 1) 
print(df.head())
#df_test = df_test.loc[:, 'Age':'WFNS grade']
#print(df_test.head())
print(df.info())

   Age  Sex  ...  Outcome at Discharge  Outcome at 3 months
0   56    1  ...                     3                    1
1   54    1  ...                     5                    5
2   66    1  ...                     5                    5
3   46    1  ...                     5                    5
4   67    0  ...                     5                    5

[5 rows x 9 columns]
   Age  Sex  Hypertension  Daibetes  WFNS grade  Outcome at 3 months
0   56    1             0         1           1                    1
1   54    1             0         1           1                    5
2   66    1             0         1           2                    5
3   46    1             0         0           1                    5
4   67    0             1         1           2                    5
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 6 columns):
Age                    627 non-null int64
Sex                    627 non-null int64
Hypertension    

In [0]:
def prepare_data_for_model(raw_dataframe, 
                           target_columns, 
                           drop_first = False, 
                           make_na_col = True):
    
    # dummy all categorical fields 
    dataframe_dummy = pd.get_dummies(raw_dataframe, columns=target_columns, 
                                     drop_first=drop_first, 
                                     dummy_na=make_na_col)
    return (dataframe_dummy)

In [49]:
num_attribs = ['Age']
cat_attribs = ['Sex','Hypertension','Daibetes','WFNS grade']
df_train_ready = prepare_data_for_model(df,cat_attribs)
#df_sub_ready = prepare_data_for_model(df_test,cat_attribs)
df_train_ready.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 16 columns):
Age                    627 non-null int64
Outcome at 3 months    627 non-null int64
Sex_0.0                627 non-null uint8
Sex_1.0                627 non-null uint8
Sex_nan                627 non-null uint8
Hypertension_0.0       627 non-null uint8
Hypertension_1.0       627 non-null uint8
Hypertension_nan       627 non-null uint8
Daibetes_0.0           627 non-null uint8
Daibetes_1.0           627 non-null uint8
Daibetes_nan           627 non-null uint8
WFNS grade_1.0         627 non-null uint8
WFNS grade_2.0         627 non-null uint8
WFNS grade_3.0         627 non-null uint8
WFNS grade_4.0         627 non-null uint8
WFNS grade_nan         627 non-null uint8
dtypes: int64(2), uint8(14)
memory usage: 18.5 KB


In [0]:
X = df_train_ready.loc[:, 'Age':'WFNS grade_nan']
y = df_train_ready.loc[:,'Outcome at 3 months']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=y)

In [0]:
num_attribs = ['Age']
cat_attribs = ['Sex','Hypertension','Daibetes','WFNS grade']

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
from sklearn.base import TransformerMixin #gives fit_transform method for free
class MyLabelBinarizer(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelBinarizer(*args, **kwargs)
    def fit(self, x, y=0):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        return self.encoder.transform(x)

In [0]:
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('num_scaler',MinMaxScaler()),
])

In [0]:
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('label_binarizer', OneHotEncoder()),
])

In [0]:
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [0]:
#X_train_prep = full_pipeline.fit_transform(X_train)
#y_train = full_pipeline.fit_transform(y_train)
#df_sub_prep = full_pipeline.fit_transform(df_test)

In [57]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
lr.score(X_test, y_test)

0.7380952380952381